# Bootstrap models

- Create sample of estimates with selection based on the probality of error

In [52]:
library(haven)
library(sdazar)
library(brms)
options(mc.cores = parallel::detectCores()-2)

Loading required package: Rcpp
Loading required package: ggplot2
Loading 'brms' package (version 2.5.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').
Run theme_set(theme_default()) to use the default bayesplot theme.

Attaching package: ‘brms’

The following object is masked from ‘package:sdazar’:

    mi



In [5]:
# create sample of estimates (10 for testing)

seed = 102330
df = data.table(read_dta('../data/FOR_BOOTSTRAP_FINAL_NOV_19_2018.dta'))

country_labels = c("Argentina", "Bolivia", "Brazil", "Chile", "Colombia", 
                   "Costa_Rica", "Cuba", "Dominican_Republic", "Ecuador", 
                   "El_Salvador", "Guatemala", "Honduras", "Mexico", "Nicaragua", 
                   "Panama", "Paraguay", "Peru", "Uruguay", "Venezuela")

df[, ctry := factor(ctry, labels=country_labels)]

In [25]:
# head(df)

In [8]:
df = melt(df, id_vars=c('ctry', 'year'), measure = patterns("^Ex", "^pr"), value.name = c("le", "pr"))
df = df[!is.na(le)]

setorder(df, ctry, year)

df[, spr := pr / sum(pr), by=.(ctry, year)]
df = df[year>=1900]

df[, N := .N, by=.(ctry,year)]

set.seed = seed
samples = df[,.SD[ sample(.N, 100, replace=TRUE, prob=pr)], by = .(ctry,year)]

samples[, sample_index := 1:.N, by=.(ctry, year)]